In [ ]:
!pip install PyPDF2
!pip install textract
import numpy as np
import pandas as pd
import re
import pickle
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
import PyPDF2
import textract

In [ ]:
def cleanResume(txt):
    cleanText = re.sub(r'http\S+\s', ' ', txt)
    cleanText = re.sub(r'RT|cc', ' ', cleanText)
    cleanText = re.sub(r'#\S+\s', ' ', cleanText)
    cleanText = re.sub(r'@\S+', '  ', cleanText)
    cleanText = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub(r'\s+', ' ', cleanText)
    return cleanText
df = pd.read_csv("/content/UpdatedResumeDataSet.csv")
df['Resume'] = df['Resume'].apply(lambda x: cleanResume(x))

le = LabelEncoder()
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(df['Resume'])
requiredText = tfidf.transform(df['Resume'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(requiredText, df['Category'], test_size=0.2, random_state=42)

In [ ]:
clf = OneVsRestClassifier(SVC(kernel='linear'))  # Use linear kernel for text classification
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(kernel='linear'))

In [ ]:
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))
pickle.dump(clf, open('clf.pkl', 'wb'))


In [ ]:
df.Category.unique()

array([ 6, 12,  0,  1, 24, 16, 22, 14,  5, 15,  4, 21,  2, 11, 18, 20,  8,
       17, 19,  7, 13, 10,  9,  3, 23])

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    pdf_text = ""
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page in pdf_reader.pages:
                pdf_text += page.extract_text()
    except Exception as e:
        return str(e)
    return pdf_text

In [ ]:
def predict_category_from_pdf(pdf_path):
    # Load the trained classifier
    clf = pickle.load(open('clf.pkl', 'rb'))

    # Extract text from the PDF resume
    text = extract_text_from_pdf(pdf_path)

    # Clean the input resume
    cleaned_resume = cleanResume(text)
    #print(cleaned_resume)

    # Transform the cleaned resume using the trained TfidfVectorizer
    input_features = tfidf.transform([cleaned_resume])

    # Make the prediction using the loaded classifier
    prediction_id = clf.predict(input_features)[0]

    #category mapping
    category_mapping = {
    15: "Java Developer",
    23: "Testing",
    8: "DevOps Engineer",
    20: "Python Developer",
    24: "Web Designing",
    12: "HR",
    13: "Hadoop",
    3: "Blockchain",
    10: "ETL Developer",
    18: "Operations Manager",
    6: "Data Science",
    22: "Sales",
    16: "Mechanical Engineer",
    1: "Arts",
    7: "Database",
    11: "Electrical Engineering",
    14: "Health and fitness",
    19: "PMO",
    4: "Business Analyst",
    9: "DotNet Developer",
    2: "Automation Testing",
    17: "Network Security Engineer",
    21: "SAP Developer",
    5: "Civil Engineer",
    0: "Advocate",
}

    # Map category ID to category name
    category_name = category_mapping.get(prediction_id, "Unknown")

    return category_name

# Example usage:
pdf_path = "E:\MCA\mini project\Blockchain Developer sample.pdf"
predicted_category = predict_category_from_pdf(pdf_path)
print("Predicted Category:", predicted_category)

UnpicklingError: ignored

In [ ]:
import joblib
modelname="clf.pkl"
joblib.dump(clf,modelname)
from google.colab import files
files.download(modelname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import joblib
modelname="tfidf.pkl"
joblib.dump(tfidf,modelname)
from google.colab import files
files.download(modelname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>